# Model Server

In [1]:
#!python -m pip install v3io

In [2]:
# nuclio: ignore
import nuclio

In [3]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel
%nuclio config spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [4]:
%%nuclio cmd -c
pip install v3io

In [5]:
import os
from cloudpickle import load
import numpy as np
from typing import List
from datetime import datetime

In [6]:
import json
import socket
import v3io.dataplane

v3io_client = v3io.dataplane.Client()
hostname = None

def split_path(mntpath=''):
    if mntpath[0] == '/':
        mntpath = mntpath[1:]
    paths = mntpath.split('/')
    container = paths[0]
    subpath = ''
    if len(paths) > 1:
        subpath = mntpath[len(container):]
    return container, subpath
 

def init_stream(stream_path, shards=1):
    global hostname
    hostname = socket.gethostname()
    container, stream_path = split_path(stream_path)
    response = v3io_client.create_stream(container=container,
                                         path=stream_path, 
                                         shard_count=shards,
                                         raise_for_status=v3io.dataplane.RaiseForStatus.never)
    response.raise_for_status([409, 204])
    print(response.__dict__)
    
def push_to_stream(stream_path, data):
    records = [{'data': json.dumps(rec)} for rec in data]
    container, stream_path = split_path(stream_path)
    response = v3io_client.put_records(container=container,
                                       path=stream_path, 
                                       records=records)
    
    #for response_record in response.output.records:
    #    print(response_record.__dict__)

In [7]:
import mlrun
class ClassifierModel(mlrun.runtimes.MLModelServer):
    def __init__(self, name: str, model_dir: str, classifier = None):
        super().__init__(name)
        self.model_dir = model_dir
        self.log_stream = os.environ.get('INFERENCE_STREAM', '')
        if self.log_stream:
            init_stream(self.log_stream)
        if classifier:
            self.classifier = classifier
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        if self.model_dir.endswith('.pkl'):
            model_file = self.model_dir
        else:
            for file in os.path.listdir(self.model_dir):
                if file.endswith('.pkl'):
                    model_file = os.pth.join(self.model_dir, file)
                    break
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        start = datetime.now()
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            resp = result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")
        
        if self.log_stream:
            data = {'request': body, 'resp': resp, 
                    'class': 'ClassifierModel', 
                    'model': self.name,
                    'host': hostname,
                    'when': str(start), 
                    'microsec': (datetime.now()-start).microseconds}
            push_to_stream(self.log_stream, [data])

        return resp

In [8]:
# nuclio: end-code

## test models locally and deploy

The sklearn-project generated 3 models that will be deployed in the server project `sklearn-servers`

### test locally

In [9]:
import cloudpickle as cp
path = '/User/artifacts/sklearn_classifier/models'

from sklearn.datasets import load_iris
iris = load_iris()

x = iris['data'].tolist()
y = iris['target']

for model in os.listdir(path):
    if model.endswith("pkl"):
        
        my_server = ClassifierModel('classifier', model_dir=os.path.join(path, model))
        my_server.load()

        a = my_server.predict({"instances": x})

        assert len(a)==150

###  deploy servers

In [10]:
from mlrun import new_model_server
from mlrun import mount_v3io, mlconf
import os

for model in os.listdir(mlconf.artifact_path + '/sklearn_classifier/models'):
    if model.endswith("pkl"):
        model = model.split('.')[-2]
        fn = new_model_server(f'iris-{model}', model_class='ClassifierModel')
        fn.spec.description = f"generic sklearn {model} model server"
        fn.metadata.categories = ['serving', 'models', f'{model}']
        fn.metadata.labels = {'author': 'yaronh'}
        fn.save()
        #print(fn.to_yaml())
        fn.export(f"{model}/function.yaml")
        fn.apply(mount_v3io())
        fn.set_envs({f"SERVING_MODEL_iris_{model}": mlconf.artifact_path + f"/sklearn_classifier/models/{model}.pkl",
                     'INFERENCE_STREAM': 'users/admin/tststream'})
        fn.deploy(project='sklearn-servers')

[mlrun] 2020-04-30 21:47:15,890 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-30 21:47:15,921 function spec saved to path: AdaBoostClassifier/function.yaml
[mlrun] 2020-04-30 21:47:15,922 deploy started
[nuclio] 2020-04-30 21:47:28,123 (info) Build complete
[nuclio] 2020-04-30 21:47:39,335 done updating sklearn-servers-iris-adaboostclassifier, function address: 3.135.130.246:31273
[mlrun] 2020-04-30 21:47:40,893 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-30 21:47:40,931 function spec saved to path: LGBMClassifier/function.yaml
[mlrun] 2020-04-30 21:47:40,932 deploy started
[nuclio] 2020-04-30 21:47:43,031 (info) Build complete
[nuclio] 2020-04-30 21:47:50,380 (info) Function deploy complete
[nuclio] 2020-04-30 21:47:50,387 done updating sklearn-servers-iris-lgbmclassifier, function address: 3.135.130.246:30547
[mlrun] 2020-04-30 21:47:51,872 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-30 21:47:5

In [11]:
import mlrun
mlrun.__version__

'0.4.7'

In [15]:

mlconf.images_tag

'0.4.7-py37'

from the image:

```
>>> import mlrun
>>> mlrun.__version__
'0.4.7'
>>> 
```